In [1]:
# Structured Streaming is a high-level API for Stream Processing.
# The same operations in batch mode can run in a streaming fashion. This can reduce latency and allow for incremental processing.

# This example uses Retail dataset, it is time-series data, is arranged by-day, is provided in Spark Definitive guide git repo.
# Creating the static dataframe on the data and capture the Schema.

staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("dbfs:/data/retail-data/by-day/*.csv")

In [2]:
# Registering the Dataframe as Table.
staticDataFrame.createOrReplaceTempView("retail_data")
# Assign the dataframe Schema to staticSchema variable.
staticSchema = staticDataFrame.schema
# Checking the Schema
staticSchema

Out[ 4 ]: StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,TimestampType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,DoubleType,true),StructField(Country,StringType,true)))

In [3]:
# Limiting the number of partitions created after shuffle. by-default 200 partitions will be created.
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [4]:
# Creating the Streaming DataFrame to process the streaming retail-dataset.

streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("dbfs:/data/retail-data/by-day/*.csv")

# Schema is manadatory to be specified for Streaming Dataframe.
# maxFilesPerTrigger is specified as 1, just to demonstrate the Streaming behavior to limit the number of files read at once. In production, you may not specify this option.

In [5]:
# Checking if the streamingDataFrame is a Streaming DF

streamingDataFrame.isStreaming

Out[ 8 ]: True

In [6]:
# Finding the customers who made the highest purchases during 1 day.
# This involves grouping the customers by InvoiceDate and calculating the purchase amount by quantity and unit price.
from pyspark.sql.functions import col, window

purchaseByCustomerPerDay = streamingDataFrame.selectExpr("CustomerId", "(Quantity * UnitPrice) as total_cost", "InvoiceDate")\
                          .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
                          .sum("total_cost")

In [7]:
# The above operation is a lazy operation, we need to call a streaming action to start the execution of Data flow.
# Streaming actions are different from our conventional static action because we’re going to be populating data somewhere instead of just calling something like count.
# calling an action to output the data to In-Memory table, that will get updated for each trigger.

purchaseByCustomerPerDay.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

Out[ 14 ]: <pyspark.sql.streaming.StreamingQuery at 0x7f35fa10c310>

In [8]:
# The Stream is started and output data will be updated to 'customer_purchases' table.
# We can query the 'customer_purchases' table using spark.sql queries.

spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """).show()

# Notice the composition of table changes as we read in more data! With each file, the results might or might not be changing based on the data, because we’re grouping customers.
# You may see few records with null customerId, because the dataset contains the null values.

+----------+--------------------+------------------+
CustomerId| window| sum(total_cost)|
+----------+--------------------+------------------+
 17450.0|[2011-09-20 00:00...| 71601.44|
 null|[2011-11-14 00:00...| 55316.08|
 null|[2011-11-07 00:00...| 42939.17|
 null|[2011-03-29 00:00...| 33521.39999999998|
 null|[2011-12-08 00:00...|31975.590000000007|
 18102.0|[2011-09-15 00:00...|31661.540000000005|
 null|[2010-12-21 00:00...|31347.479999999938|
 18102.0|[2011-10-21 00:00...| 29693.82|
 18102.0|[2010-12-07 00:00...| 25920.37|
 14646.0|[2011-10-20 00:00...|25833.559999999994|
 null|[2010-12-10 00:00...|25399.560000000012|
 null|[2010-12-17 00:00...|25371.769999999768|
 null|[2011-11-25 00:00...|24148.069999999992|
 null|[2011-11-29 00:00...|23744.250000000055|
 12415.0|[2011-06-15 00:00...| 23426.81000000001|
 null|[2010-12-06 00:00...|23395.099999999904|
 null|[2011-08-30 00:00...| 23032.59999999993|
 null|[2010-12-03 00:00...| 23021.99999999999|
 15749.0|[2011-01-11 00:00...| 22998.4|
 18102.0|[2011-10-03 00:00...| 22429.69|
+----------+--------------------+------------------+
only showing top 20 rows

In [9]:
# In the above example, we have output the data to In-Memory table.Alternatively we can output the data to the console as well, as below:

#purchaseByCustomerPerHour.writeStream
#    .format("console")
#    .queryName("customer_purchases_2")
#    .outputMode("complete")
#    .start()